## Implementing Recommendation engine

In [1]:
from surprise import Dataset
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\SAINADH/.surprise_data/ml-100k


In [2]:
from surprise.model_selection import train_test_split 
train_data,test_data = train_test_split(data,test_size = 0.2)

# ITEM-ITEM COLLABORATIVE FILTERING


In [3]:
from surprise import KNNWithMeans
knn = KNNWithMeans(k = 30, sim_options = {'name': 'pearson_baseline', 'user_based': False})
knn.fit(train_data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [4]:
y_pred = knn.test(test_data)
from surprise import accuracy
accuracy.rmse(y_pred, verbose = True)

RMSE: 0.9224


0.9223981768622053

In [5]:
knn.test([('45', '800', 5.0)])

[Prediction(uid='45', iid='800', r_ui=5.0, est=2.976423173081955, details={'actual_k': 16, 'was_impossible': False})]

# USER-USER COLLABORATIVE FILTERING

In [6]:
knn = KNNWithMeans(k = 30 , sim_options = {'name': 'pearson_baseline', 'user_based' : True})
knn.fit(train_data)
y_pred = knn.test(test_data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [7]:
accuracy.rmse(y_pred, verbose = True)

RMSE: 0.9393


0.9392585057438554

In [8]:
knn.predict('196','302',4.0, verbose =True)

user: 196        item: 302        r_ui = 4.00   est = 4.29   {'actual_k': 30, 'was_impossible': False}


Prediction(uid='196', iid='302', r_ui=4.0, est=4.290147191277123, details={'actual_k': 30, 'was_impossible': False})

# MATRIX FACTORIZATION

In [9]:
from surprise.model_selection import GridSearchCV
from surprise import SVD

param_grid = {'n_factors': [110, 120],
              'n_epochs': [10, 5, 11],
              'lr_all': [0.001, 0.003, 0.005, 0.008],
              'reg_all': [0.08, 0.1]}
grid_search = GridSearchCV(SVD, param_grid, measures = ['RMSE'], cv = 5)
grid_search.fit(data)
grid_search.best_params['rmse']

{'n_factors': 120, 'n_epochs': 11, 'lr_all': 0.008, 'reg_all': 0.08}

In [11]:
b_params = grid_search.best_estimator['rmse']
from surprise.model_selection import cross_validate
cross_validate(b_params,data,measures = ['RMSE'], cv =5)

{'test_rmse': array([0.93566616, 0.93492263, 0.93917967, 0.93967303, 0.93525832]),
 'fit_time': (2.8429040908813477,
  2.487269878387451,
  2.354973554611206,
  2.3614661693573,
  2.371615409851074),
 'test_time': (0.1770336627960205,
  0.14763522148132324,
  0.11279058456420898,
  0.1426072120666504,
  0.1616065502166748)}

In [12]:
svd = SVD(n_factors=120, n_epochs=11, lr_all=0.008, reg_all=0.08)
svd.fit(train_data)
y_pred = svd.test(test_data)
accuracy.rmse(y_pred)

RMSE: 0.9340


0.9339849726916551